In [ ]:
!pip install nesmdb==0.1.8
!pip install pretty_midi
!wget http://deepyeti.ucsd.edu/cdonahue/nesmdb/nesmdb_midi.tar.gz
!tar xvfz nesmdb_midi.tar.gz

In [20]:
import os
import pretty_midi

four_instruments = []

for files in os.walk('nesmdb_midi/train'):
    for file in files[2]:
        if file.endswith('.mid'):
            #print(file)
            try:
                midi_data = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + file)
                if len(midi_data.instruments) > 3:
                    four_instruments.append(file)
                    print(file)
                    print(midi_data.instruments)
            except:
                print("error on file: " + file)
            

108_Famista_90_10_11Unknown2.mid
[Instrument(program=80, is_drum=False, name="p1"), Instrument(program=81, is_drum=False, name="p2"), Instrument(program=38, is_drum=False, name="tr"), Instrument(program=121, is_drum=True, name="no")]
178_Ironsword_Wizards_amp_WarriorsII_12_13FireElemental.mid
[Instrument(program=80, is_drum=False, name="p1"), Instrument(program=81, is_drum=False, name="p2"), Instrument(program=38, is_drum=False, name="tr"), Instrument(program=121, is_drum=True, name="no")]
195_KonamiWaiWaiWorld_18_19FinalStageBossBGM.mid
[Instrument(program=80, is_drum=False, name="p1"), Instrument(program=81, is_drum=False, name="p2"), Instrument(program=38, is_drum=False, name="tr"), Instrument(program=121, is_drum=True, name="no")]
050_ChaosWorld_20_21Chaos.mid
[Instrument(program=80, is_drum=False, name="p1"), Instrument(program=81, is_drum=False, name="p2"), Instrument(program=38, is_drum=False, name="tr"), Instrument(program=121, is_drum=True, name="no")]
100_Falsion_05_06FlyingH

In [21]:
len(four_instruments)

2416

In [49]:
pm = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + four_instruments[100])

In [54]:
four_instruments[100]

'135_GanbareGoemonGaiden2_TenkanoZaih__87_88SupriseEmergency.mid'

In [53]:
print("Time estimate")
print(pm.estimate_tempo())
print("inst 0")
print(pm.instruments[0].name)
print(pm.instruments[0].notes)
print("inst 1")
print(pm.instruments[1].name)
print(pm.instruments[1].notes)
print("inst 2")
print(pm.instruments[2].name)
print(pm.instruments[2].notes)
print("inst 3")
print(pm.instruments[3].name)
print(pm.instruments[3].notes)

Time estimate
236.49635036496375
inst 0
p1
[Note(start=0.000884, end=0.200227, pitch=41, velocity=8), Note(start=0.400272, end=0.400408, pitch=41, velocity=8), Note(start=0.400476, end=0.532993, pitch=77, velocity=8), Note(start=0.532993, end=0.582902, pitch=76, velocity=5), Note(start=0.582902, end=0.699388, pitch=77, velocity=5), Note(start=0.699388, end=0.749297, pitch=76, velocity=5), Note(start=0.749297, end=0.865782, pitch=77, velocity=5), Note(start=0.865782, end=0.915692, pitch=76, velocity=5), Note(start=0.915692, end=1.032177, pitch=77, velocity=5), Note(start=1.032177, end=1.082086, pitch=76, velocity=5), Note(start=1.082086, end=1.198571, pitch=77, velocity=5), Note(start=1.198571, end=1.248481, pitch=76, velocity=5), Note(start=1.248481, end=1.364943, pitch=77, velocity=5), Note(start=1.364943, end=1.414875, pitch=76, velocity=5), Note(start=1.414875, end=1.531338, pitch=77, velocity=5), Note(start=1.531338, end=1.581270, pitch=76, velocity=5), Note(start=1.581270, end=1.5

In [ ]:
#custom mapping
custom_mapping = 

In [34]:
pm = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + four_instruments[0])
#get piano roll
piano_roll = pm.get_piano_roll(times=pm.get_beats())
#get tempo changes and their times
times, tempi = pm.get_tempo_changes()

print(pm.get_beats())

print(pm.get_downbeats())

onsets = pm.instruments[0].get_onsets()
print(onsets)
print(len(onsets))

[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5
 21.  21.5 22.  22.5 23.  23.5 24.  24.5 25.  25.5]
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5
 21.  21.5 22.  22.5 23.  23.5 24.  24.5 25.  25.5]
[1.31519274e-03 1.00521542e-01 2.00362812e-01 3.00204082e-01
 4.00408163e-01 6.99659864e-01 1.09900227e+00 1.59820862e+00
 1.69789116e+00 1.79773243e+00 1.89757370e+00 2.29702948e+00
 2.69637188e+00 2.69639456e+00 3.59501134e+00 3.69446712e+00
 3.79424036e+00 3.89428571e+00 3.99401361e+00 4.09394558e+00
 4.19374150e+00 4.29356009e+00 4.39358277e+00 4.49315193e+00
 4.59292517e+00 4.69297052e+00 4.79269841e+00 4.89263039e+00
 4.99247166e+00 5.09229025e+00 5.19242630e+

In [27]:
for track in four_instruments[0:2]:
    print(track)
    midi_data = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + track)
    #print(midi_data)
    print(midi_data.get_tempo_changes())
    print(midi_data.estimate_tempo())
    for instrument in midi_data.instruments:
        print(instrument)
        print(instrument.notes)

108_Famista_90_10_11Unknown2.mid
(array([0.]), array([120.]))
158.20509562708787
Instrument(program=80, is_drum=False, name="p1")
[Note(start=0.001315, end=0.100522, pitch=81, velocity=3), Note(start=0.100522, end=0.200363, pitch=83, velocity=3), Note(start=0.200363, end=0.300204, pitch=84, velocity=3), Note(start=0.300204, end=0.400408, pitch=86, velocity=3), Note(start=0.400408, end=0.665805, pitch=88, velocity=3), Note(start=0.699660, end=0.965306, pitch=86, velocity=3), Note(start=1.099002, end=1.364649, pitch=91, velocity=3), Note(start=1.598209, end=1.697891, pitch=91, velocity=3), Note(start=1.697891, end=1.797732, pitch=89, velocity=3), Note(start=1.797732, end=1.897574, pitch=88, velocity=3), Note(start=1.897574, end=2.263175, pitch=86, velocity=3), Note(start=2.297029, end=2.562676, pitch=83, velocity=3), Note(start=2.696372, end=2.696395, pitch=83, velocity=3), Note(start=2.696395, end=2.962018, pitch=84, velocity=3), Note(start=3.595011, end=3.694467, pitch=84, velocity=3),